In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
DATA_DIR = "./ASL Dataset" # <-- change to the path where your `train` and `test` folders live
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [ ]:
IMG_SIZE = (128, 128) # image size used for training (change to 224 for larger models)
BATCH_SIZE = 32
EPOCHS = 30
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
NUM_CLASSES = None

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='training'
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='validation'
)

In [ ]:
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(f"Found {NUM_CLASSES} classes:", class_names)

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential([
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.08),
layers.RandomZoom(0.08),
])

In [ ]:
inputs = keras.Input(shape=(*IMG_SIZE, 3))


x = data_augmentation(inputs)
# rescale to [0,1]
x = layers.Rescaling(1./255)(x)


x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)


x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)


x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)


x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)


outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)


model = keras.Model(inputs, outputs, name='asl_cnn')

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("asl_best.h5", save_best_only=True, monitor='val_accuracy', mode='max'),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])


epochs_range = range(len(acc))


plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='train acc')
plt.plot(epochs_range, val_acc, label='val acc')
plt.legend()
plt.title('Accuracy')


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='train loss')
plt.plot(epochs_range, val_loss, label='val loss')
plt.legend()
plt.title('Loss')
plt.show()

In [ ]:
model.save('asl_saved_model')
print('Saved model to asl_saved_model/')

In [ ]:


# %%
from tensorflow.keras.preprocessing import image


def predict_image(img_path, model, img_size=IMG_SIZE, class_names=None):
img = image.load_img(img_path, target_size=img_size)
arr = image.img_to_array(img)
arr = arr / 255.0
arr = np.expand_dims(arr, axis=0)
preds = model.predict(arr)
top_idx = np.argmax(preds[0])
prob = preds[0][top_idx]
label = class_names[top_idx] if class_names is not None else str(top_idx)
return label, float(prob)


# load best weights if saved by checkpoint
if os.path.exists('asl_best.h5'):
print('Loading best weights from asl_best.h5')
model.load_weights('asl_best.h5')


# loop through images in test dir (non-recursive)
if os.path.isdir(TEST_DIR):
test_images = [os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR)
if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


for p in test_images:
label, prob = predict_image(p, model, class_names=class_names)
print(f"{os.path.basename(p)} -> {label} ({prob:.3f})")
else:
print('Test directory not found at', TEST_DIR)


# %% [markdown]
# ## Using NumPy to build a manual mini-batch for debugging (optional)
#
# If you want to quickly debug a single batch using NumPy arrays (e.g., for unit tests), here's how to take a batch from the dataset and convert to NumPy:


# %%
for images_batch, labels_batch in train_ds.take(1):
images_np = images_batch.numpy()
labels_np = labels_batch.numpy()
print('images_np.shape =', images_np.shape)
print('labels_np.shape =', labels_np.shape)
break


# %% [markdown]
# ## Next steps / improvements
# - Replace the simple CNN with a MobileNetV2 / EfficientNet backbone for much better accuracy on small datasets (use `tf.keras.applications`).
# - Use class weights if your dataset is imbalanced.
# - Add stratified splitting or ensure each class appears in validation set.
# - Consider using `tf.data` pipelines loading images directly and applying augmentation for very large datasets.
# - If you plan to deploy on mobile, test the TFLite conversion and quantize the model.


# EOF